In [64]:
# Cell 1: Setup and user-defined variables
import os

# Set these paths accordingly
data_dir = "/data/tmpA/andrem/linger/preprocessed_obj/AML12_DX_MO_scdart"            # Directory with RNA.txt, ATAC.txt, label.txt
datadir = "/data/benchmarks/andrem/linger"              # Directory with bulk GRN data (will look for data_bulk/)
outdir = "/data/tmpA/andrem/linger/out/AML12_DX_MO_scdart"            # Output directory
method = "LINGER"                        # or 'baseline'
genome = "hg38"
network = "cell population"
celltype = "all"
activation = "ReLU"
cuda_device = "0"

# Set CUDA device
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device
print(f"Using CUDA device(s): {cuda_device}")
os.makedirs(outdir, exist_ok=True)


Using CUDA device(s): 0


In [3]:
# Cell 2: Imports
import pandas as pd
from scipy.sparse import csc_matrix
import anndata
import scanpy as sc
#import muon as mu

from LingerGRN.preprocess import get_adata, preprocess
from LingerGRN.pseudo_bulk import pseudo_bulk
import LingerGRN.LINGER_tr as LINGER_tr
import LingerGRN.LL_net as LL_net
from LingerGRN.TF_activity import regulon, heatmap_cluster, master_regulator, box_comp


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [14]:
# Cell 3: Load and merge input data
def clean_barcode_columns(columns):
    return columns.str.split('___').str[0].str.replace('-1$', '', regex=True)

print("Loading input data...")
label = pd.read_csv(os.path.join(data_dir, 'label.txt'), sep='\t')
RNA = pd.read_csv(os.path.join(data_dir, 'RNA.txt'), sep='\t', index_col=0)
ATAC = pd.read_csv(os.path.join(data_dir, 'ATAC.txt'), sep='\t', index_col=0)

#TODO meter isto correcto no preparate input => dps de verficar se isto é o correto
RNA.columns = clean_barcode_columns(RNA.columns)
ATAC.columns = clean_barcode_columns(ATAC.columns)
label = pd.read_csv(os.path.join(data_dir, 'label.txt'), sep='\t')
# Clean the "barcode" column and rename it to "barcode_use"
label['barcode_use'] = label['barcode'].str.split('___').str[0].str.replace('-1$', '', regex=True)
# Drop the original "index" column if it exists
label = label.drop(columns=['index'], errors='ignore')
# Reorder columns: "label", "barcode_use"
label = label[['label', 'barcode_use']]


matrix = csc_matrix(pd.concat([RNA, ATAC], axis=0).values)
features = pd.DataFrame(RNA.index.tolist() + ATAC.index.tolist(), columns=[1])
features[2] = ['Gene Expression'] * RNA.shape[0] + ['Peaks'] * ATAC.shape[0]
barcodes = pd.DataFrame(RNA.columns.values, columns=[0])


Loading input data...


In [16]:
# Cell 4: Create AnnData objects and filter
adata_RNA, adata_ATAC = get_adata(matrix, features, barcodes, label)

sc.pp.filter_cells(adata_RNA, min_genes=200)
sc.pp.filter_genes(adata_RNA, min_cells=3)
sc.pp.filter_cells(adata_ATAC, min_genes=200)
sc.pp.filter_genes(adata_ATAC, min_cells=3)

# Keep only intersected barcodes
selected = list(set(adata_RNA.obs['barcode']) & set(adata_ATAC.obs['barcode']))
adata_RNA = adata_RNA[[b in selected for b in adata_RNA.obs['barcode']]]
adata_ATAC = adata_ATAC[[b in selected for b in adata_ATAC.obs['barcode']]]

print(f"Filtered cells: {adata_RNA.n_obs} shared barcodes")


Trying to modify attribute `.obs` of view, initializing view as actual.
Trying to modify attribute `.obs` of view, initializing view as actual.
Trying to modify attribute `.obs` of view, initializing view as actual.
Trying to modify attribute `.obs` of view, initializing view as actual.


Filtered cells: 6074 shared barcodes


In [ ]:
from LingerGRN.pseudo_bulk import *

samplelist = list(set(adata_ATAC.obs['sample'].values))  # unique sample IDs
singlepseudobulk = adata_RNA.obs['sample'].nunique() ** 2 > 100

TG_pseudobulk = pd.DataFrame([])
RE_pseudobulk = pd.DataFrame([])

for tempsample in samplelist:
    adata_RNAtemp = adata_RNA[adata_RNA.obs['sample'] == tempsample]
    adata_ATACtemp = adata_ATAC[adata_ATAC.obs['sample'] == tempsample]
    
    TG_pseudobulk_temp, RE_pseudobulk_temp = pseudo_bulk(adata_RNAtemp, adata_ATACtemp, singlepseudobulk)

    RE_pseudobulk_temp[RE_pseudobulk_temp > 100] = 100

    TG_pseudobulk = pd.concat([TG_pseudobulk, TG_pseudobulk_temp], axis=1)
    RE_pseudobulk = pd.concat([RE_pseudobulk, RE_pseudobulk_temp], axis=1)

# Save outputs
import os
if not os.path.exists('data/'):
    os.mkdir('data/')

#adata_ATAC.write('data/adata_ATAC.h5ad')
#adata_RNA.write('data/adata_RNA.h5ad')
pd.DataFrame(adata_ATAC.var['gene_ids']).to_csv(os.path.join(outdir, "Peaks.txt"), header=None, index=None)

TG_pseudobulk = TG_pseudobulk.fillna(0)
RE_pseudobulk = RE_pseudobulk.fillna(0)
TG_pseudobulk.to_csv('data/TG_pseudobulk.tsv', sep='\t')
RE_pseudobulk.to_csv('data/RE_pseudobulk.tsv', sep='\t')

print(f"✅ Pseudo-bulk completed: TG {TG_pseudobulk.shape}, RE {RE_pseudobulk.shape}")


Received a view of an AnnData. Making a copy.
Received a view of an AnnData. Making a copy.
Received a view of an AnnData. Making a copy.
Received a view of an AnnData. Making a copy.
Trying to modify attribute `.obs` of view, initializing view as actual.
Trying to modify attribute `.obs` of view, initializing view as actual.


✅ Pseudo-bulk completed: TG (17360, 229), RE (205003, 229)


In [33]:
from LingerGRN.preprocess import preprocess
import inspect

print(inspect.getfile(LINGER_tr.training))

/home/andrem/GRN-project/workflow/.snakemake/conda/76d78891a26d915bcb4ceb8a190f4007_/lib/python3.10/site-packages/LingerGRN/LINGER_tr.py


In [ ]:
# Cell 6: Preprocess + train model
GRNdir = os.path.join(datadir, 'data_bulk/')
preprocess(TG_pseudobulk, RE_pseudobulk, GRNdir, genome, method, outdir)



Mapping gene expression...
Generate TF expression...
Generate RE chromatin accessibility...
Generate TF binding...


100%|██████████| 23/23 [12:14<00:00, 31.93s/it]


Generate Index...


100%|██████████| 12802/12802 [00:27<00:00, 466.77it/s]


TypeError: training() missing 1 required positional argument: 'species'

In [44]:

import importlib
importlib.reload(LINGER_tr)


<module 'LingerGRN.LINGER_tr' from '/home/andrem/GRN-project/workflow/.snakemake/conda/76d78891a26d915bcb4ceb8a190f4007_/lib/python3.10/site-packages/LingerGRN/LINGER_tr.py'>

In [45]:
LINGER_tr.training(GRNdir, method, outdir, activation, "hg38")

chr1


100%|██████████| 1318/1318 [42:47<00:00,  1.95s/it]


chr2


100%|██████████| 853/853 [26:10<00:00,  1.84s/it]


chr3


...

100%|██████████| 117/117 [03:46<00:00,  1.94s/it]


chr22


100%|██████████| 319/319 [11:32<00:00,  2.17s/it]


chrX


100%|██████████| 421/421 [11:26<00:00,  1.63s/it]


In [58]:
import importlib
importlib.reload(LingerGRN.LL_net)


<module 'LingerGRN.LL_net' from '/home/andrem/GRN-project/workflow/.snakemake/conda/76d78891a26d915bcb4ceb8a190f4007_/lib/python3.10/site-packages/LingerGRN/LL_net.py'>

In [ ]:
# Cell 7: Build Cell Population GRN
LL_net.TF_RE_binding(GRNdir, adata_RNA, adata_ATAC, genome, method, outdir)



Generating cellular population TF binding strength ...


  0%|          | 0/23 [00:00<?, ?it/s]

Generating cellular population TF binding strength for chr1


  4%|▍         | 1/23 [09:45<3:34:31, 585.08s/it]

Generating cellular population TF binding strength for chr2


...

Generating cellular population TF binding strength for chr22


 96%|█████████▌| 22/23 [1:34:12<04:00, 240.44s/it]

Generating cellular population TF binding strength for chrX


  0%|          | 0/23 [00:17<?, ?it/s]


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy.core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([numpy.core.multiarray.scalar])` or the `torch.serialization.safe_globals([numpy.core.multiarray.scalar])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn mor

In [59]:
LL_net.cis_reg(GRNdir, adata_RNA, adata_ATAC, genome, method, outdir)


100%|██████████| 23/23 [03:19<00:00,  8.68s/it]


In [65]:
LL_net.trans_reg(GRNdir, method, outdir, genome)

Generate trans-regulatory netowrk ...


  0%|          | 0/23 [00:00<?, ?it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fbdff983880>>
Traceback (most recent call last):
  File "/home/andrem/GRN-project/workflow/.snakemake/conda/76d78891a26d915bcb4ceb8a190f4007_/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
 17%|█▋        | 4/23 [00:25<02:01,  6.40s/it]


KeyboardInterrupt: 

In [ ]:
tfb_potential = os.path.join(outdir, 'cell_population_TF_RE_binding.txt')
re_tg = os.path.join(outdir, 'cell_population_cis_regulatory.txt')
tf_tg = os.path.join(outdir, 'cell_population_trans_regulatory.txt')

